In [2]:
import pandas as pd

In [3]:
pd.__version__

'1.4.2'

In [4]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [5]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [6]:
import sklearn

In [7]:
sklearn.__version__

'1.0.2'

In [8]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() // 60

In [9]:
df['duration'].std()

42.594499992966355

In [10]:
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

In [11]:
(len(filtered_df) / len(df)) * 100

98.18303711466737

In [12]:
pickup_ids = df['PULocationID'].astype(str).unique()
dropoff_ids = df['DOLocationID'].astype(str).unique()

print("Pickup 고유값 수:", len(pickup_ids))
print("Dropoff 고유값 수:", len(dropoff_ids))

# 전체 one-hot 열 수 예상치
combined_ids = set(['PULocationID=' + i for i in pickup_ids]) | set(['DOLocationID=' + i for i in dropoff_ids])
print("예상 총 feature 수:", len(combined_ids))

Pickup 고유값 수: 257
Dropoff 고유값 수: 261
예상 총 feature 수: 518


In [13]:
from sklearn.feature_extraction import DictVectorizer

# 필요한 컬럼 선택 및 문자열로 변환
df_subset = df[['PULocationID', 'DOLocationID']].astype(str)

# 데이터프레임을 사전 리스트로 변환
records = df_subset.to_dict('records')

# DictVectorizer로 원-핫 인코딩
dv = DictVectorizer()
X = dv.fit_transform(records)

# 피처 매트릭스의 차원(컬럼 수) 확인
num_columns = X.shape[1]
print(f"문제 4: 피처 매트릭스의 컬럼 수: {num_columns}")

문제 4: 피처 매트릭스의 컬럼 수: 518


이전 단계에서 만든 feature matrix (pickup & dropoff location을 one-hot encoding한 것)를 사용하여  
선형 회귀 모델을 훈련시키세요. 목표 변수는 duration입니다.

- 기본 파라미터로 선형 회귀 모델 학습
- 학습 데이터에 대한 RMSE(Root Mean Squared Error) 계산

질문: 훈련 데이터의 RMSE는 얼마인가요?

In [14]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np


# 피처 준비 (PULocationID와 DOLocationID만 사용)
df_subset = filtered_df[['PULocationID', 'DOLocationID']].astype(str)
records = df_subset.to_dict('records')

# 원-핫 인코딩
dv = DictVectorizer()
X_train = dv.fit_transform(records)

# 응답 변수
y_train = filtered_df['duration'].values

# 선형 회귀 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 학습 데이터에 대한 예측
y_pred = model.predict(X_train)

# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"문제 5: 학습 데이터에서의 RMSE: {rmse:.2f}")

문제 5: 학습 데이터에서의 RMSE: 7.69


In [15]:
test = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [16]:
test['duration'] = (test['tpep_dropoff_datetime'] - test['tpep_pickup_datetime']).dt.total_seconds() / 60
test = test[(test['duration'] >= 1) & (test['duration'] <= 60)]

# 검증 데이터 피처 준비
df_val_subset = test[['PULocationID', 'DOLocationID']].astype(str)
val_records = df_val_subset.to_dict('records')

# 동일한 DictVectorizer로 피처 변환
X_val = dv.transform(val_records)
y_val = test['duration'].values
ss
# 검증 데이터에 대한 예측
y_pred = model.predict(X_val)

# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"문제 6: 검증 데이터에서의 RMSE: {rmse:.2f}")

문제 6: 검증 데이터에서의 RMSE: 7.85
